In [1]:
import gymnasium
import highway_env
# import finite-mdp
# from finite_mdp.envs import finite_mdp_env

%load_ext tensorboard
import sys
from tqdm.notebook import trange
# !pip install tensorboardx gym pyvirtualdisplay
# doesn't work cause not linux
# !apt-get install -y xvfb ffmpeg
# !git clone https://github.com/Farama-Foundation/HighwayEnv.git 2> /dev/null
# !git clone https://github.com/eleurent/finite-mdp.git 2> /dev/null
sys.path.insert(0, '/content/HighwayEnv/scripts/')
# from utils import record_videos, show_videos



In [ ]:
import gymnasium as gym
import random
import highway_env
from collections import defaultdict



env = gym.make("highway-fast-v0", config={
#    "observation": {
#     "type": "Kinematics",
#     "vehicles_count": 5,
#     "features": ["presence", "x", "y", "vx", "vy"],
#     "features_range": {
#         "x": [-100, 100],
#         "y": [-100, 100],
#         "vx": [-20, 20],
#         "vy": [-20, 20]
#     },
#     "grid_size": [[-27.5, 27.5], [-27.5, 27.5]],
#     "grid_step": [5, 5],
#     "absolute": False,
# },
 "observation": {
    "type": "TimeToCollision",
    "horizon": 5,
    # "lanes_count":2,
    # "lanes_count":2,
},
#   "action":{
#     "type": "DiscreteAction",
# }
}
)

# obs, info = env.reset()


class FiniteMarkovDecision():
    def __init__(self,env):
        self.finite_mdp = self.is_finite_mdp(env)
        self.states = env.observation_space
        if self.finite_mdp:
            self.mpd = env.mdp
        elif not self.finite_mdp:
            try:
                self.mdp = env.unwrapped.to_finite_mdp()
            except AttributeError:
                raise TypeError("not finite mdp")
            
        self.env = env
        self.obs, self.info = env.reset() 
        self.state_action_value = env.action_space
        self.policy = defaultdict()


    def bellman(self):

        # print(self.state_action_value.n)
        gamma = 0.9
        episodes = 2
        episode_count = 0
        while episodes > 0:
            done = True
            total_reward = 0
            while not done:
                i = random.randrange(0, self.state_action_value.n)
                next_state, reward, done, truncated, info = self.env.step(i)
                total_reward += (reward*gamma*episode_count)
                episode_count += 1
            state = tuple(self.obs)
            if self.policy[state] < total_reward:
                self.policy[self.obs] = i
            episodes -= 1
            
            


    def is_finite_mdp(self,env):
        try:
            finite_mdp = __import__("finite_mdp.envs.finite_mdp_env")
            if isinstance(env.unwrapped, finite_mdp.envs.finite_mdp_env.FiniteMDPEnv):
                return True
        except (ModuleNotFoundError, TypeError):
            return False


# obs, info = env.reset()
# print(obs)
# x = env.to_finite_mdp()
# print(env.__dict__.keys())
# print(env.action_space)
finite_mdp = FiniteMarkovDecision(env)
finite_mdp.bellman()

(array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32), array([[0. , 0. , 1. , 1. , 0.5],
       [0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. ]], dtype=float32))


TypeError: unhashable type: 'numpy.ndarray'